In [1]:
import os
import numpy as np
from skimage import io
from skimage.transform import resize
from skimage.feature import graycomatrix, graycoprops
from skimage.color import rgb2gray, rgb2hsv
from skimage import img_as_ubyte
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report

In [2]:
# Definir la ruta de la carpeta que contiene las carpetas de objetos
base_folder = "F://Users//Admin//Documents//Programas//Objetos"

# Inicializar listas para almacenar imágenes y etiquetas
X = []
y = []
scale = 8
img_width = int(1920/scale)
img_height = int(1080/scale)

# Recorrer cada carpeta de objetos
for object_folder in os.listdir(base_folder):
    if os.path.isdir(os.path.join(base_folder, object_folder)):
        # Recorrer cada imagen en la carpeta
        for filename in os.listdir(os.path.join(base_folder, object_folder)):
            img_path = os.path.join(base_folder, object_folder, filename)
            
            # Cargar la imagen y redimensionarla
            rgb = io.imread(img_path)
            rgb_resized = resize(rgb, (img_height, img_width), anti_aliasing=True)
            
            # Agregar la imagen y la etiqueta a las listas
            X.append(rgb_resized)
            y.append(object_folder)

# Convertir listas a matrices numpy
X = np.array(X)
y = np.array(y)

# One-hot encode the labels
le = LabelEncoder()
y = le.fit_transform(y)

# Shuffle the data
X, y = shuffle(X, y, random_state=42)

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
X_train = X_train / 255.0
X_test = X_test / 255.0

In [3]:
# Cambia la capa de entrada para manejar imágenes en escala de grises
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(len(np.unique(y)), activation='softmax'))

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Accuracy en el conjunto de prueba: {test_acc}')
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)
print(classification_report(y_test, y_pred, target_names=le.classes_))

Epoch 1/10
63/63 [==============================] - 80s 1s/step - loss: 1.2663 - accuracy: 0.4975 - val_loss: 0.4805 - val_accuracy: 0.7560
Epoch 2/10
63/63 [==============================] - 87s 1s/step - loss: 0.1983 - accuracy: 0.9410 - val_loss: 0.0706 - val_accuracy: 0.9800
Epoch 3/10
63/63 [==============================] - 86s 1s/step - loss: 0.0420 - accuracy: 0.9910 - val_loss: 0.0239 - val_accuracy: 0.9960
Epoch 4/10
63/63 [==============================] - 91s 1s/step - loss: 0.0133 - accuracy: 0.9980 - val_loss: 0.0134 - val_accuracy: 1.0000
Epoch 5/10
63/63 [==============================] - 97s 2s/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 6/10
63/63 [==============================] - 95s 2s/step - loss: 0.5938 - accuracy: 0.9130 - val_loss: 0.0365 - val_accuracy: 0.9980
Epoch 7/10
63/63 [==============================] - 96s 2s/step - loss: 0.0112 - accuracy: 0.9990 - val_loss: 0.0047 - val_accuracy: 1.0000
Epoch 8/10
63/63 [==